In [1]:
import cassandra

In [12]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

In [13]:
# Creating the keyspace (~ database)
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS data_modelling
        WITH REPLICATION =
            {
                'class' : 'SimpleStrategy',
                'replication_factor' : 1
            }
    """)
except Exception as e:
    print(e)

In [14]:
# Connect to the keyspace
try:
    session.set_keyspace('data_modelling')
except Exception as e:
    print(e)

<h2>Questions for our data</h2>
<h4>1. Give me all albums of a particular artist</h4>
SELECT * FROM MUSIC_LIBRARY WHERE ARTIST_NAME = 'THE BEATLES'

In [15]:
query = 'CREATE TABLE IF NOT EXISTS music_library '
query += '(YEAR INT, CITY TEXT, ARTIST_NAME TEXT, ALBUM_NAME TEXT, PRIMARY KEY (ARTIST_NAME, ALBUM_NAME, CITY))'
try:
    session.execute(query)
except Exception as e:
    print(e)

In [16]:
query = 'INSERT INTO music_library (YEAR, CITY, ARTIST_NAME, ALBUM_NAME)'
query += ' VALUES (%s, %s, %s, %s)'
try:
    session.execute(query, (1970, 'New York', 'The Beatles', 'Let It Be'))
    session.execute(query, (1970, 'Bangalore', 'The Beatles', 'I Will Win'))
except Exception as e:
    print(e)

In [18]:
# Create table
try:
    rows = session.execute("SELECT * FROM music_library WHERE ARTIST_NAME = 'The Beatles'")
except Exception as e:
    print(e)
for row in rows:
    print(', '.join([row.artist_name, str(row.year), row.album_name, row.city]))

The Beatles, 1970, I Will Win, Bangalore
The Beatles, 1970, Let It Be, New York


In [11]:
query = "DROP TABLE MUSIC_LIBRARY;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="unconfigured table music_library"


In [43]:
session.shutdown()
cluster.shutdown()